In [1]:
import numpy as np
from scipy.integrate import solve_bvp
import matplotlib.pyplot as plt

In [2]:
def obstacle(x,y,W1=1,r=(1,1),c=(0,0)):
    '''
    Define an area that will represent an obstacle
    
    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    ellipse = ((x - c[0])**2/r[0] + (y - c[1])**2/r[1])**20 + 1

    return W1 / ellipse

def obstacle_dx(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    x derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40* W1 * (x-c[0])*(circle)**19
    denom = r[0]*((circle)**20 + 1)**2

    return numer / denom

def obstacle_dy(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    y derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40 * W1 * (y-c[1])*(circle)**19
    denom = r[1]*((circle)**20 + 1)**2

    return numer / denom

In [110]:
def prob2():
    '''
    Find the optimal path for the fixed time horizon problem
    '''
    alpha, beta, gamma, delta = 5, 5, 1, 1
    cx, cy = 0.5, 0.5
    r = 0.01
    start_x, start_y, start_vx, start_vy, start_theta = 0.1, 0.5, 0, 0, 0
    end_x, end_y, end_vx, end_vy, end_theta = 0.9, 0.5, 0, 0, 0
    lambda_1, lambda_2 = 3, 3
    def ode(t, y):
#         #y0=s1, y1=s2, y2=s3, y3=s4, y4=p1, y5=p2, y6=p3, y7=p4
#         inside = (y[0]-cx)**2 / rx + (y[1]-cy)**2 / ry
#         p1 = -2 * lambda_ * W1 * (y[0]-cx) * (inside)**(lambda_-1) / (rx * (inside**lambda_ + 1)**2)
#         p2 = -2 * lambda_ * W1 * (y[1]-cy) * (inside)**(lambda_-1) / (ry * (inside**lambda_ + 1)**2)
        #y0=s1, y1=s2, y2=s3, y3=s4, y4=s5, y5=p1, y6=p2, y7=p3, y8=p4, y9=p5
        u0 = -(y[7]*np.cos(y[4])+y[8]*np.sin(y[4]))/(2*gamma)
        u1 = -y[9]/(2*delta)
        denom = (y[0]-cx)**2+(y[1]-cy)**2-r**2
        p1 = beta * lambda_1 * (1/y[0]**(lambda_1+1)-1/(1-y[0])**(lambda_1+1)) - 2 * lambda_2 * alpha * (y[0]-cx) / denom**(lambda_2+1)
        p2 = beta * lambda_1 * (1/y[1]**(lambda_1+1)-1/(1-y[1])**(lambda_1+1)) - 2 * lambda_2 * alpha * (y[1]-cy) / denom**(lambda_2+1)
        return [y[2], y[3], u0*np.cos(y[4]), u0*np.sin(y[4]), u1, 
                p1, p2, -y[5], -y[6], u0*(y[7]*np.sin(y[4])-y[8]*np.sin(y[4]))]
    bc = lambda ya, yb: [ya[0]-start_x, ya[1]-start_y, ya[2]-start_vx, ya[3]-start_vy, ya[4]-start_theta,
                         yb[0]-end_x, yb[1]-end_y, yb[2]-end_vx, yb[3]-end_vy, yb[4]-end_theta]
    t_steps = 10001
    t = np.linspace(0, 1, t_steps)
    y0 = np.random.rand(10,t_steps)
    sol = solve_bvp(ode, bc, t, y0, max_nodes=300000)
    plt.plot(sol.y[0], sol.y[1], 'r', label="optimal path")
    # Plot obstacle
    domain_x = np.linspace(0, 1, 1001)
    domain_y = np.linspace(0, 1, 1001)
    X, Y = np.meshgrid(domain_x, domain_y)
    z = obstacle(X, Y, 1, (r, r), (cx, cy))
    plt.contour(X, Y, z)
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    return sol

In [111]:
sol = prob2()

MemoryError: Unable to allocate 206. MiB for an array with shape (270000, 10, 10) and data type float64

In [ ]:
sol